In [6]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
import matplotlib as mpl
from matplotlib.dates import (YEARLY, DateFormatter,
                              rrulewrapper, RRuleLocator, drange)
import numpy as np
import os
import datetime
import pandas as pd
import xarray as xr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from PIL import Image

root_dir='/gpfs/fs1/work/hongli/sharp/basins/tayprk_huc12/scripts_hongli/'
os.chdir(root_dir)
output_folder='step4_plot_summa_output_day_png'
outputfile = 'summa_output_hru.png'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

sim_file='../output/gmet_3hr/output_output_day.nc'
var_names = ['pptrate_mean', 'airtemp_mean', 'scalarCanopyWat', 'scalarSWE', 'scalarAquiferStorage', 'scalarTotalSoilWat',
            'scalarSurfaceRunoff_mean', 'scalarSoilBaseflow_mean', 'scalarAquiferBaseflow_mean', 
             'scalarTotalET_mean', 'scalarTotalRunoff_mean']
var_units = ['kg m-2 s-1', 'K', 'kg m-2', 'kg m-2','m', 'kg m-2', 'm s-1', 'm s-1', 'm s-1', 'kg m-2 s-1', 'm s-1']
t_vname = 'time'

hru_indexs = [0, 1, 2]
time_format='%Y-%m-%d'
# utc_local_shift = -6 #hr shift from UTC to local
time_step = 24 #hr
time_num_perday = 24/time_step

plot_start_date_summary=['2007-10-02']
plot_end_date_summary = ['2017-09-30']
# formatter = DateFormatter('%m/%d/%y')
formatter = DateFormatter('%Y') #'%b-%y'
dpi_value = 100
ylabels = ['Precip Rate', 'Air Temprature', 'Canopy Water', 'SWE', 'Aquifer Storage', 'Total Soil Water',
            'Surface Runoff', 'Soil Baseflow', 'Aquifer Baseflow', 'Total ET', 'Total Runoff']

# read simulated flow (m^3/s)
f = xr.open_dataset(sim_file)
var_list = f.variables
sim_time = f[t_vname].dt.strftime(time_format).values[:]

os.chdir(os.path.join(root_dir, output_folder))
for hru_index in hru_indexs:
    print('hru '+str(hru_index+1))
    hru_fig_file = os.path.join(root_dir, output_folder, 
                               outputfile.split('.')[0]+str(hru_index+1)+'.'+outputfile.split('.')[1])
    #plot each time step 
    print('plot')
    temp_fig_files = []    
    for m in range(len(plot_start_date_summary)):
        plot_start_str=plot_start_date_summary[m]
        plot_end_str=plot_end_date_summary[m]
#         print(plot_start_str)

        plot_start=int(((datetime.datetime.strptime(plot_start_str,time_format)-datetime.datetime.strptime(sim_time[0],time_format)).days)*time_num_perday)
        plot_len=int(((datetime.datetime.strptime(plot_end_str,time_format)-datetime.datetime.strptime(plot_start_str,time_format)).days)*time_num_perday)

        plot_start_date=datetime.datetime.strptime(plot_start_str,time_format)
        t=[plot_start_date+datetime.timedelta(hours=hr*time_step) for hr in range(0,plot_len)]
        t_plot=[mpl.dates.date2num(hr) for hr in t[0:plot_len]]  

        temp_fig_file = 'image_'+str(m+1)+'.png'
        temp_fig_files.append(temp_fig_file)
        
        row_num=len(var_names) 
        fig, ax = plt.subplots(row_num,1)
        fig.set_figwidth(6.5*1.3) #190mm
        fig.set_figheight(6.5*0.2*row_num)

        plot_var_count = 0        
        for k in range(row_num):
            var_index = k
            var_name = var_names[var_index]

            if var_name in var_list:                    

                var_value = f[var_name].values[:]
                var_unit = var_units[var_names.index(var_name)]
                plot_alpha = chr(ord('a')+plot_var_count)
                plot_var_count = plot_var_count+1

                if var_unit == 'kg m-2 s-1':
                    var_value = var_value*86400 # flux from kg/m^2/s to mm/day
                    var_unit = 'mm/d'
                elif var_unit == 'kg m-2':
                    var_unit = 'mm'
                    var_value = var_value
                elif var_unit == 'm s-1':
                    var_unit = 'mm/d'
                    var_value = var_value*1000*86400 # water depth from /s to /day
                elif var_unit == 'K':
                    var_unit = '$^\circ$C'
                    var_value = var_value - 273.15 # temp from K to C
                elif var_unit == 'm':
                    var_unit = 'mm'
                    var_value = var_value*1000 # temp from K to C
                
                subplot_left = 0.25
                subplot_base_bottom = (row_num-k)*1.0/row_num
                subplot_width = 0.65
                subplot_hight = 0.83/row_num
                ax[k].set_position([subplot_left, subplot_base_bottom-subplot_hight, subplot_width, subplot_hight]) #[left, bottom, width, height] 

                var_value_plot = var_value[plot_start+0:plot_start+plot_len, hru_index] 
                ax[k].plot_date(t_plot, var_value_plot, 'bo-', linewidth=1.0, markersize=0.0, alpha=0.8)   

                ylabel = ylabels[k]+'\n'+'('+var_unit+')'
                fig.text(0.5*subplot_left, subplot_base_bottom-0.5*subplot_hight, ylabel, ha='center', va='center', rotation='horizontal')
#                     ax[k].set_ylabel(ylabel, fontsize='medium', rotation='horizontal', horizontalalignment='center')
                ax[k].yaxis.set_tick_params(labelsize='medium')

                ax[k].xaxis.set_major_formatter(formatter)
                ax[k].xaxis.set_tick_params(labelsize='medium')#rotation=30, 
                
                if not k==row_num-1:
                    ax[k].xaxis.set_ticklabels([])

                del var_value, var_value_plot

        fig.savefig(temp_fig_file, dpi=dpi_value)
        plt.close(fig)                    
        
    # concatenate into one figure    
    print('save')
    widths = []
    heights = []
    for temp_fig_file in temp_fig_files:
        im = Image.open(temp_fig_file)
        widths.append(im.width)
        heights.append(im.height)

    max_width = max(widths)
    total_height = sum(heights)
    new_im = Image.new('RGB', (max_width, total_height))

    x_offset = 0
    for temp_fig_file in temp_fig_files:
        im = Image.open(temp_fig_file)    
        new_im.paste(im, (0,x_offset))
        x_offset += im.size[1]
    new_im.save(hru_fig_file)

    for file in os.listdir(os.getcwd()):
        if file.startswith('image'):
            os.remove(file)
    [os.remove(file) for file in os.getcwd() if file.startswith('image')]   

f.close()
os.chdir(root_dir)


hru 1
plot
save
hru 2
plot
save
hru 3
plot
save


In [42]:
row_num =11
for k in range(row_num):
#     print(k,(row_num-k-1)*1.0/row_num)
    print(k,(k+1)*(1.0/row_num))

0 0.09090909090909091
1 0.18181818181818182
2 0.2727272727272727
3 0.36363636363636365
4 0.4545454545454546
5 0.5454545454545454
6 0.6363636363636364
7 0.7272727272727273
8 0.8181818181818182
9 0.9090909090909092
10 1.0


In [41]:
row_num

11